In [1]:
from datetime import datetime, date
import calendar
import pandas as pd

In [10]:
# this cell initialises the program with current pupils as of 2023-05-15 and regular pupil lengths and pupil fees

pupils = ['cleo', 'bruno', 'fabi', 'jaks', 'max', 'joe', 'george', 'clarissa', 'jasper', 'charlie', 'umaiyini', 'magali', 'julia', 'eva', 'filip', 'triantis', 'linus']

pupil_lengths = {"p0": 30,
 "p1": 30,
 "p2": 30,
 "p3": 30,
 "p4": 30,
 "p5": 30,
 "p6": 30,
 "p7": 30,
 "p8": 30,
 "p9": 60,
 "p10": 30,
 "p11": 30,
 "p12": 30,
 "p13": 20,
 "p14": 30,
 "p15": 30,
 "p16": 30}

lesson_length_fees = {20: 18.0, 30: 25.0, 60: 45.0, 90: 70.0}

pupil_siblings = {"p0": False,
 "p1": False,
 "p2": False,
 "p3": False,
 "p4": "p5",
 "p5": "p4",
 "p6": False,
 "p7": "p8",
 "p8": "p7",
 "p9": False,
 "p10": False,
 "p11": "p12",
 "p12": "p11",
 "p13": False,
 "p14": False,
 "p15": False,
 "p16": False}

pupil_codes = {}

for idx, pupil in enumerate(pupils):
    pupil_codes["p" + str(idx)] = pupil
    
reverse_pupil_codes = {value: key for key, value in pupil_codes.items()} # this has the pupil's name as the key, and the code as the value

pupil_data = {'Codes' : ["p0", "p1", "p2", "p3", "p4", "p5", 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16'],
        'Name': ['cleo', 'bruno', 'fabi', 'jaks', 'max', 'joe', 'george', 'clarissa', 'jasper', 'charlie', 'umaiyini', 'magali', 'julia', 'eva', 'filip', 'triantis', 'linus'],
        'Length': [30, 30, 30, 30, 30, 30, 30, 30, 30, 60, 30, 30, 30, 20, 30, 30, 30],
        'Sibling?': [False, False, False, False, "p5", "p4", False, "p8", "p7", False, False, "p12", "p11", False, False, False, False]}
pupil_data_frame = pd.DataFrame(pupil_data)
print(pupil_data_frame)
pupil_data_frame.to_csv('output.csv', index=False)


   Codes      Name  Length Sibling?
0     p0      cleo      30    False
1     p1     bruno      30    False
2     p2      fabi      30    False
3     p3      jaks      30    False
4     p4       max      30       p5
5     p5       joe      30       p4
6     p6    george      30    False
7     p7  clarissa      30       p8
8     p8    jasper      30       p7
9     p9   charlie      60    False
10   p10  umaiyini      30    False
11   p11    magali      30      p12
12   p12     julia      30      p11
13   p13       eva      20    False
14   p14     filip      30    False
15   p15  triantis      30    False
16   p16     linus      30    False


In [3]:
lesson_keys = ["L0"] # capital L to avoid confusion with number 1
lesson_dates = {}
lesson_pupils = {} # this stores the pupil code, not the pupil name
lesson_lengths = {}
lesson_fees = {}

In [56]:
# functions used to input data about a specific lesson

def get_date(prompt=""):
    valid = False

    print(prompt)
    
    while not valid:
        try:
            user_input = input()
            
            if user_input == "cancel": # the user can cancel any time, in which case it returns False
                return False
            
            else:
                date_parts = user_input.split('/') # splits the user input eg. '12/3/23' -> (12, 3, 23)
                if len(date_parts) > 3: # eg '12/3/4/32'
                    print("Input valid date")
                    continue
                day, month = map(int, date_parts[:2])
                if len(date_parts) == 2: # if input omitted year
                    year = datetime.now().year
                else:
                    year = int(date_parts[2])
                    if year < 100: # this turns eg '23' into '2023' for consistency
                        year += 2000
                return date(year, month, day)
                valid = True
        except:
            print("Input valid date")

def get_pupil(prompt=""): #NB this will return pupil code, not pupil name
    valid = False

    while not valid:
        user_input = input(prompt)
        
        if user_input == "cancel": # the user can cancel any time, in which case it returns False
            return False
        
        matching_pupils = [pupil for pupil in pupils if pupil.startswith(user_input.lower())] # makes a list of pupils whose name begins with user input. This allows for inputing eg 'ha' instead of 'harry'
        
        if matching_pupils: # checks if there are any matching pupils
            if len(matching_pupils) == 1: # will only accept there being one matching pupil to avoid ambiguity
                valid = True
                return reverse_pupil_codes[matching_pupils[0].lower()]
            else:
                print("Multiple pupils found. Did you mean:") # shows user if there are more than one matching pupils.
                print(", ".join(matching_pupils))

        else:
            print(user_input + " is not a valid pupil")

def get_length(pupil_code, is_sibling_lesson): # returns lesson length
    valid = False

    while not valid:
        try:
            if not is_sibling_lesson:
                user_input = input(str(pupil_lengths[pupil_code]) + " minute lesson?") # suggests usual lesson length

                if user_input == "cancel": # the user can cancel any time, in which case it returns False
                    return False

                elif user_input == "":
                    return pupil_lengths[pupil_code]
            else:
                user_input = input(str(pupil_lengths[pupil_code] + pupil_lengths[pupil_siblings[pupil_code]]) + " minute lesson?") # suggests usual lesson length

                if user_input == "cancel": # the user can cancel any time, in which case it returns False
                    return False

                elif user_input == "":
                    return pupil_lengths[pupil_code] + pupil_lengths[pupil_siblings[pupil_code]]

            # if user inputs a number rather than just hitting return, it accepts that as the lesson length
            length = int(user_input)

            if 10 <= length <= 120: # min and max values for lesson length in minutes
                valid = True
                return user_input
            else:
                print("lesson length must be between 10mins and 2hrs")
        except:
            print(str(user_input) + " is not a valid lesson length")

def get_fee(lesson_length): # returns lesson fee
    valid = False

    while not valid:
        try:
            user_input = input(f"paid £{length_fees[lesson_length]:.2f}?") # suggests usual lesson fee
            
            if user_input == "cancel": # the user can cancel any time, in which case it returns False
                return False
        
            elif user_input == "":
                return length_fees[lesson_length]
            
            # if user inputs a number rather than just hitting return, it accepts that as the lesson fee
            length = int(user_input)
            
            if 10 <= length <= 90: # min and max values for lesson fee in GBP
                valid = True
                return user_input
            else:
                print("lesson fee must be between £10 and £90")
        except:
            print(str(user_input) + " is not a valid lesson fee")

def lesson_report(): # returns False if the user inputs cancel at any time

    confirmed = False
    is_sibling_lesson = False
    
    while not confirmed:
        lesson_key = "L" + str((int(lesson_keys[-1][1:]) + 1))

        lesson_pupil = get_pupil("Who were you teaching? ") # NB this returns the pupil code, or an empty string if the user inputs 'cancel'
        if not lesson_pupil: # ie if the user inputted 'cancel'
            return False
        elif pupil_siblings[lesson_pupil]:
            user_input = input("Did you also teach " + pupil_codes[pupil_siblings[lesson_pupil]].title() + "? (y/n)").lower()
            if user_input == "y":
                is_sibling_lesson = True
            elif user_input == "cancel":
                return False

        lesson_date = get_date("When was this lesson")
        if not lesson_date: # ie if the user inputted 'cancel'
            return False

        lesson_length = get_length(lesson_pupil, is_sibling_lesson)
        if not lesson_length: # ie if the user inputted 'cancel'
            return False

        lesson_fee = get_fee(lesson_length)
        if not lesson_fee: # ie if the user inputted 'cancel'
            return False

        print("\nHere are the lesson details for you to confirm:")
        if not is_sibling_lesson:
            print("pupil        : " + pupil_codes[lesson_pupil].title())
        else:
            print("pupils       : " + pupil_codes[lesson_pupil].title() + " and " + pupil_codes[pupil_siblings[lesson_pupil]].title())
         
        print("date         : " + lesson_date.strftime("%A %d/%m/%Y\n") +
              "lesson length: " + str(lesson_length) + " minutes\n" +
              f"fee          : £{lesson_fee:.2f}")

        user_input = input("Confirm? (y/n)")
        if user_input == "y":
            confirmed = True
            lesson_keys.append(lesson_key)
            if not is_sibling_lesson:
                lesson_pupils[lesson_key] = lesson_pupil
            else:
                lesson_pupils[lesson_key] = (lesson_pupil, pupil_siblings[lesson_pupil])
            lesson_dates[lesson_key] = lesson_date
            lesson_lengths[lesson_key] = lesson_length
            lesson_fees[lesson_key] = lesson_fee
        elif user_input == "cancel":
            return False

def print_ten_most_recent_lessons():
    print("Here are your ten most recent lessons:")
    print("date      pupil     length   fee")
    sorted_dates = sorted(lesson_dates, key=lambda x: x[1])[:9]
    for lesson_code in sorted_dates:
        lesson_date = lesson_dates[lesson_code].strftime("%d/%m/%y")
        print(f"{lesson_date:<10}{pupil_codes[lesson_pupils[lesson_code]].title():<10}{lesson_lengths[lesson_code]:<3}mins  £{lesson_fees[lesson_code]:.2f}")


In [50]:
# other functions

def get_month_and_year(): # returns a tuple (month, year). Month value is 0 if user only inputs a year
    valid = False
    while not valid:
        user_input = input()
        if " " in user_input:
            date_parts = user_input.split(" ")
            if len(date_parts) == 2:
                #gets the month
                try:
                    if len(date_parts[0]) == 3:
                        month = datetime.strptime(date_parts[0], "%b").month
                    else:
                        month = datetime.strptime(date_parts[0], "%B").month
                except:
                    print("Input valid time")
                    continue
                try:
                    if int(date_parts[1]) < 100:
                        year = 2000 + int(date_parts[1])
                        valid = True
                    else:
                        year = int(date_parts[1])
                        valid = True
                except:
                    print("Input valid time")
                    continue
        else:
            month = 0
            try:
                if int(user_input) < 100:
                    year = 2000 + int(user_input)
                    valid = True
                else:
                    year = int(user_input)
                    valid = True
            except:
                print("Input valid time")
                continue

        return month, year
        

In [ ]:
user_input = input("Hello Ed. Would you like to report a new lesson? (y/n) ").lower()

while True:
    if user_input == "y":
        lesson_report()
        user_input = input("Report another lesson? (y/n)")
    else:
        print("Here are some options:\nLesson history (h)\nAdd new pupil (a)\nback (b)")
        user_input = input().lower()
        if user_input == "h":
            total_earned = 0
            total_hours = 0
            matching_lessons = []
            
            print_ten_most_recent_lessons()
            
            print("View lessons from a certain time (t) or from a certain pupil (p)?")
            user_input = input().lower()
            
            if user_input == "t":
                print("input time")
                month, year = get_month_and_year()
                if month == 0:
                    time_formatted = str(year)
                    for lesson_code, date in lesson_dates.items():
                        if date.year == year:
                            matching_lessons.append(lesson_code)
                    total_lessons = len(matching_lessons)
                else:
                    time_formatted = calendar.month_name[month] + " " + str(year)
                    for lesson_code, date in lesson_dates.items():
                        if date.year == year and date.month == month:
                            matching_lessons.append(lesson_code)
                    total_lessons = len(matching_lessons)

                for lesson_code in matching_lessons:
                    total_earned += lesson_fees[lesson_code]
                    total_hours += lesson_lengths[lesson_code]
                
                input(f"In {time_formatted} you earned £{total_earned:.2f}\npress enter to continue")
            elif user_input == "p":
                pupil = get_pupil("input pupil")
                for lesson_code, pupil_code in lesson_pupils.items():
                    if pupil == pupil_code:
                        matching_lessons.append(lesson_code)
                total_lessons = len(matching_lessons)
                
                for lesson_code in matching_lessons:
                    total_earned += lesson_fees[lesson_code]
                    total_hours += lesson_lengths[lesson_code]
                
                input(f"You have earned £{total_earned:.2f} teaching {pupil_codes[pupil].title()}.\npress enter to continue")
        
        

Hello Ed. Would you like to report a new lesson? (y/n) y
Who were you teaching? Br
When was this lesson
6/5
Input valid date
8/9
Input valid date
3/6/23
Input valid date
2 may
Input valid date
Input valid date
Input valid date


In [1]:
#lesson_keys
lesson_dates
#lesson_pupils
#lesson_lengths
#lesson_fees

NameError: name 'lesson_dates' is not defined

In [2]:
pupil_data_frame.to_csv('output.csv', index=False)

      Name  Age      City
0    Alice   25  New York
1      Bob   32    London
2  Charlie   18     Paris
3     Dave   47     Tokyo


Program for helping keep track of piano lessons

For each lesson it should store a unique lesson key, the pupil, the date, the time and the fee
	Stored in a series of dictionaries - the key is the lesson code, the value is the name/date/length/fee etc.

	Each pupil should have a dictionary as well, with all their previous lesson keys stored in them.

Info to store about each pupil - unique pupil key, name, normal lesson length, normal lesson fee, as well as a record of all previous lessons']



On opening program:
1: Greet Ed, say would you like to report new lesson?
2: if yes, then report lesson, goto 1
3: else, 

Here are some options
View lessons (v)
Add new pupil (a)
